# Entregável 3 de Visão Computacional e Robótica


Entrega até 12/03 ao fim do atendimento

Pode ser feito **em duplas**

Nesta semana vamos trabalhar com um assunto extremamente atual: reconhecimento de objetos e rastreamento


Referências:

[https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/](https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/)

[https://github.com/iArunava/YOLOv3-Object-Detection-with-OpenCV/](https://github.com/iArunava/YOLOv3-Object-Detection-with-OpenCV/)

[https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/](https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/)

Ouça a explicacão do professor sobre rastreamento e deteção

### 1. Executar os três exemplos

Há três exemplos: `mobilenet_detection`, `yolov3_detection` e `tracking`.

Os dois primeiros são reconhecedores de objetos, e o último é de rastreamento


Um dos arquivos abaixo precisa ser baixado e salvo nas pasta  `yolov3_detection/yolov3-coco` .

[https://www.dropbox.com/s/013ogt2bhwfzxwb/yolov3.weights?dl=0](https://www.dropbox.com/s/013ogt2bhwfzxwb/yolov3.weights?dl=0) ou [https://pjreddie.com/media/files/yolov3.weights](https://pjreddie.com/media/files/yolov3.weights)

## 2. Identificar objeto

Escolha um dos projetos: `yolov3_detection` ou `mobilenet_detection` para basear seu código. 

Neste projeto, escolha uma categoria de objetos que o reconhecedor reconhece. Diga aqui qual foi sua escolha

<center> <h3>Projeto escolhido: yolov3_detection </h3></center>
<center><h3> Categoria escolhida: bottle </h3></center>

## 3. Rastrear

Implemente a seguinte funcionalidade: sempre que o objeto identificado em (2) estiver presente por mais que 5 frames seguidos, pare de rodar a identificação e comece a rastreá-lo.

Faça um vídeo com a  demonstração funcionalidade e mostre o link ao professor

In [ ]:
import numpy as np
import argparse
import cv2
import subprocess
import time
import os
from yolo_utils import infer_image, show_image
import imutils
from imutils.video import VideoStream
from imutils.video import FPS

FLAGS = []


"""

Adapted from:

https://github.com/iArunava/YOLOv3-Object-Detection-with-OpenCV/blob/master/yolo.py#L154

"""
def print_categories(boxes, confidences, classids, labels):
	lista = list()
	""" 
	Nota do Miranda: 
		Função estratégica para modificar/ usar o Yolo
	"""
	for i in range(len(classids)):
		lista.append(labels[classids[i]])
	return lista


def caixa(boxes, classids):
	listaboxes = list()
	for i in range(len(classids)):
		listaboxes.append(boxes[i])
	return listaboxes[0]

vid = cv2.VideoCapture(0)


########################################################################## TRACKING ##########################################################################
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video", type=str,
	help="path to input video file")
ap.add_argument("-t", "--tracker", type=str, default="kcf",
	help="OpenCV object tracker type")
args = vars(ap.parse_args())

# extract the OpenCV version info
(major, minor) = cv2.__version__.split(".")[:2]

# if we are using OpenCV 3.2 OR BEFORE, we can use a special factory
# function to create our object tracker
if int(major) == 3 and int(minor) < 3:
	tracker = cv2.Tracker_create(args["tracker"].upper())

# otherwise, for OpenCV 3.3 OR NEWER, we need to explicity call the
# approrpiate object tracker constructor:
else:
	# initialize a dictionary that maps strings to their corresponding
	# OpenCV object tracker implementations
	OPENCV_OBJECT_TRACKERS = {
		"csrt": cv2.TrackerCSRT_create,
		"kcf": cv2.TrackerKCF_create,
		"boosting": cv2.TrackerBoosting_create,
		"mil": cv2.TrackerMIL_create,
		"tld": cv2.TrackerTLD_create,
		"medianflow": cv2.TrackerMedianFlow_create,
		"mosse": cv2.TrackerMOSSE_create
	}

	# grab the appropriate object tracker using our dictionary of
	# OpenCV object tracker objects
	tracker = OPENCV_OBJECT_TRACKERS[args["tracker"]]()

# initialize the bounding box coordinates of the object we are going
# to track
objeto = None





if __name__ == '__main__':
	parser = argparse.ArgumentParser()

	parser.add_argument('-m', '--model-path',
		type=str,
		default='./yolov3-coco/',
		help='The directory where the model weights and \
			  configuration files are.')

	parser.add_argument('-w', '--weights',
		type=str,
		default='./yolov3-coco/yolov3.weights',
		help='Path to the file which contains the weights \
			 	for YOLOv3.')

	parser.add_argument('-cfg', '--config',
		type=str,
		default='./yolov3-coco/yolov3.cfg',
		help='Path to the configuration file for the YOLOv3 model.')

	parser.add_argument('-i', '--image-path',
		type=str,
		help='The path to the image file')

	parser.add_argument('-v', '--video-path',
		type=str,
		help='The path to the video file')


	parser.add_argument('-vo', '--video-output-path',
		type=str,
        default='./output.avi',
		help='The path of the output video file')

	parser.add_argument('-l', '--labels',
		type=str,
		default='./yolov3-coco/coco-labels',
		help='Path to the file having the \
					labels in a new-line seperated way.')

	parser.add_argument('-c', '--confidence',
		type=float,
		default=0.5,
		help='The model will reject boundaries which has a \
				probabiity less than the confidence value. \
				default: 0.5')

	parser.add_argument('-th', '--threshold',
		type=float,
		default=0.3,
		help='The threshold to use when applying the \
				Non-Max Suppresion')

	parser.add_argument('--download-model',
		type=bool,
		default=False,
		help='Set to True, if the model weights and configurations \
				are not present on your local machine.')

	parser.add_argument('-t', '--show-time',
		type=bool,
		default=False,
		help='Show the time taken to infer each image.')

	FLAGS, unparsed = parser.parse_known_args()

	# Download the YOLOv3 models if needed
	if FLAGS.download_model:
		subprocess.call(['./yolov3-coco/get_model.sh'])

	# Get the labels
	labels = open(FLAGS.labels).read().strip().split('\n')

	# Known categories
	print("Known categories: ", " ".join(labels))

	# Intializing colors to represent each label uniquely
	colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

	# Load the weights and configutation to form the pretrained YOLOv3 model
	net = cv2.dnn.readNetFromDarknet(FLAGS.config, FLAGS.weights)

	# Get the output layer names of the model
	layer_names = net.getLayerNames()
	layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
        


	

	count = 0
	bottlecounter = 0
	fps = None

	while True:
		ret, frame = vid.read()
		if ret != False:
			frame = imutils.resize(frame, width=600)


			height, width = frame.shape[:2]

			if bottlecounter <5 :
				if count == 0:
					frame, boxes, confidences, classids, idxs = infer_image(net, layer_names, \
										height, width, frame, colors, labels, FLAGS)
					count += 1
				else:
					frame, boxes, confidences, classids, idxs = infer_image(net, layer_names, \
										height, width, frame, colors, labels, FLAGS, boxes, confidences, classids, idxs, infer=False)
					count = (count + 1) % 6
			



			if any("bottle" in i for i in print_categories(boxes, confidences, classids, labels)):
				bottlecounter += 1


			if bottlecounter >= 5:
				objeto = tuple(caixa(boxes, classids))
				if objeto is not None:
					tracker.init(frame, objeto)
					fps = FPS().start()
					# grab the new bounding box coordinates of the object
					(success, box) = tracker.update(frame)

					# check to see if the tracking was a success
					if success:
						(x, y, w, h) = [int(v) for v in box]
						cv2.rectangle(frame, (x, y), (x + w, y + h),
							(0, 255, 0), 2)

					# update the FPS counter
					fps.update()
					fps.stop()

					# initialize the set of information we'll be displaying on
					# the frame
					info = [
						("Tracker", args["tracker"]),
						("Success", "Yes" if success else "No"),
						("FPS", "{:.2f}".format(fps.fps())),
					]

					# loop over the info tuples and draw them on our frame
					for (i, (k, v)) in enumerate(info):
						text = "{}: {}".format(k, v)
						cv2.putText(frame, text, (10, height - ((i * 20) + 20)),
							cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)


					

			cv2.imshow('webcam', frame)

			if cv2.waitKey(1) & 0xFF == ord('q'):
				break
	vid.release()
	cv2.destroyAllWindows()

In [8]:
from IPython.display import Video
Video("tracker.mp4")

## 4. Simulador

 Rode o simulador do Turtlebot (use o Waffle).  Veja o guia em [../guides/simulador_ros.md](../guides/simulador_ros.md)
 
 Documente aqui as linhas necessárias para teleop e para abrir o Rviz
 
 Faça um screenshot do seu simulação em execução

#### Teleop: rqt_image_view, roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch
#### Rviz: roslaunch turtlebot3_gazebo turtlebot3_gazebo_rviz.launch

<img src = "Teleop.png">
<img src= "Rviz.png">

## 5. Robô quadrado

Faça [este tutorial](../guides/projeto_rospython.md) de como criar um projeto Python que comanda o robô simulado.

Usando o simulador, crie um código que faça o robô fazer uma trajetória que aproxima um quadrado.

Baseie-se no código `roda.py`, construído durante o tutorial 

In [ ]:
#! /usr/bin/env python
# -*- coding:utf-8 -*-

import rospy
from geometry_msgs.msg import Twist, Vector3

v = 0.1  # Velocidade linear
w = 0.4 # Velocidade angular

if __name__ == "__main__":
    rospy.init_node("roda_exemplo")
    pub = rospy.Publisher("cmd_vel", Twist, queue_size=3)

    try:
        while not rospy.is_shutdown():

            pub.publish(Twist(Vector3(v, 0, 0), Vector3(0,0,0)))
            rospy.sleep(3.0)

            pub.publish(Twist(Vector3(0, 0, 0), Vector3(0,0,0)))
            rospy.sleep(3.0)

            pub.publish(Twist(Vector3(0, 0, 0), Vector3(0,0,w)))
            rospy.sleep(4.0)

            pub.publish(Twist(Vector3(0, 0, 0), Vector3(0,0,0)))
            rospy.sleep(3.0)



         

           
    except rospy.ROSInterruptException:
        print("Ocorreu uma exceção com o rospy")

## 6. Robô indeciso

Usando o simulador e o LIDAR simulado, faça um robô avançar quando o obstáculo bem à sua frente estiver a menos de 1.0m e recuar quando estiver a mais de 1.02 m.

Baseie-se no código `le_scan.py` e `roda.py`, desenvolvidos [durante o tutorial](../guides/projeto_rospython.md)

In [ ]:
import rospy
import numpy as np
from geometry_msgs.msg import Twist, Vector3
from sensor_msgs.msg import LaserScan


andar = False
d = 100

def scaneou(dado):
    global d
 
    d = np.amin(np.array(dado.ranges).round(decimals=2))
    
   
    


if __name__=="__main__":

    rospy.init_node("le_scan")

    velocidade_saida = rospy.Publisher("/cmd_vel", Twist, queue_size = 3 )
    recebe_scan = rospy.Subscriber("/scan", LaserScan, scaneou)



    while not rospy.is_shutdown():

        while not andar:
            print(d)
        
            if d > 1.02:
                print("Livre")
                velocidade_saida.publish(Twist(Vector3(0,0,0), Vector3(0,0,0)))
                rospy.sleep(2.0)

                andar = True

            else:
                velocidade_saida.publish(Twist(Vector3(-0.3, 0, 0), Vector3(0, 0, 0)))
                rospy.sleep(1.0) 
                velocidade_saida.publish(Twist(Vector3(0,0,0), Vector3(0,0,1)))
                rospy.sleep(2.0)
                velocidade_saida.publish(Twist(Vector3(0.3, 0, 0), Vector3(0, 0, 0)))
                rospy.sleep(2)


        while andar:

            if d > 1.02:
                velocidade_saida.publish(Twist(Vector3(0.3, 0, 0), Vector3(0, 0, 0)))
                rospy.sleep(2.0)
                     

            else:
                velocidade_saida.publish(Twist(Vector3(0,0,0), Vector3(0,0,0)))
                rospy.sleep(2.0)
                velocidade_saida.publish(Twist(Vector3(-0.3, 0, 0), Vector3(0, 0, 0)))
                rospy.sleep(1.0) 
                andar  = False 
 

